In [ ]:
import torch
from pytorch3d.io import IO
from pytorch3d.renderer import TexturesVertex

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load mesh
# We are using a mesh from the ShapeNet Dataset (https://shapenet.org/)
io = IO()
mesh = io.load_mesh("airplane.ply", include_textures=False)
mesh.textures = TexturesVertex(verts_features=torch.ones_like(mesh.verts_packed()[None]) * 0.7)
mesh = mesh.to(device)

In [ ]:
from feature_backprojection import features_from_views, DINOWrapper
from utils.geometry import pairwise_geodesic_distances_mesh
from utils.rendering import setup_renderer, sample_view_points
import torch

# Compute features
renderer = setup_renderer(device)
model = DINOWrapper(device, small=True)
render_dist = 1.1
views = sample_view_points(render_dist, 5)
geo_dists = pairwise_geodesic_distances_mesh(mesh.verts_packed().to("cpu"), mesh.faces_packed().to("cpu"))
features = features_from_views(
    renderer=renderer, model=model, mesh=mesh, views=views, render_dist=render_dist, batch_size=8,
    device=device, geo_dists=geo_dists, gaussian_sigma=0.001, only_visible=True
)

In [ ]:
from sklearn.decomposition import PCA
from pytorch3d.vis.plotly_vis import plot_scene

# Perform PCA for visualization
pca = PCA(n_components=3)
features_pca = pca.fit_transform(features.cpu().numpy())
features_pca = (features_pca - features_pca.min(axis=0)) / (features_pca.max(axis=0) - features_pca.min(axis=0))
mesh.textures = TexturesVertex(verts_features=torch.tensor(features_pca, dtype=torch.float32)[None].to(device))
plot_scene({
    "mesh": {
        "mesh": mesh,
    }
})